# Caso Práctico: Sistema Inteligente de Clasificación de Reseñas para "GastroReseñas"

**Empresa  GastroReseñas Contexto**: "GastroReseñas" es una plataforma web líder en la recopilación de opiniones sobre restaurantes. Diariamente, miles de usuarios publican reseñas sobre sus experiencias gastronómicas. Actualmente, la plataforma solo muestra una calificación de 1 a 5 estrellas.

La empresa quiere lanzar una nueva funcionalidad: un "resumen de opinión" que etiquete automáticamente cada reseña con aspectos clave mencionados por el usuario. Esto permitiría a los nuevos clientes filtrar, por ejemplo, por reseñas que hablen positivamente del "Servicio" o negativamente del "Precio".




**El Desafío**: El volumen de reseñas es demasiado grande para ser etiquetado manualmente. El equipo de "GastroReseñas" ha recopilado un conjunto de datos (un archivo .csv) donde han etiquetado manualmente 10,000 reseñas de ejemplo. El problema es que una misma reseña puede referirse a múltiples aspectos. Por ejemplo:

- Reseña: "La comida estaba deliciosa, pero el camarero fue muy lento."

- Etiquetas deseadas: [Comida_Positivo], [Servicio_Negativo]

Esto se conoce como un problema de clasificación de texto multietiqueta (multi-label classification).

## Enunciado del Problema y Tarea 

Tu objetivo es desarrollar un sistema basado en un modelo Transformer (como BERT, DistilBERT o alguno similar que hayamos dado en clase) capaz de leer el texto de una nueva reseña de restaurante y asignar todas las etiquetas que correspondan de una lista predefinida.


Conjunto de Datos: Se te proporciona un archivo gastro_reseñas_etiquetadas.csv. Las columnas relevantes son:

- id_reseña: Identificador único.

- texto_reseña: El texto completo de la opinión del usuario.

- etiqueta_comida_pos: (1 o 0)

- etiqueta_comida_neg: (1 o 0)

- etiqueta_servicio_pos: (1 o 0)

- etiqueta_servicio_neg: (1 o 0)

- etiqueta_ambiente_pos: (1 o 0)

- etiqueta_ambiente_neg: (1 o 0)

- etiqueta_precio_pos: (1 o 0)

- etiqueta_precio_neg: (1 o 0)

(Nota: Un 1 indica que la etiqueta aplica, un 0 que no aplica. Una misma fila puede tener múltiples 1s).

## Requisitos del Proyecto



Debes configurar un script o notebook de Python que implemente la solución completa utilizando la biblioteca transformers (de Hugging Face) y un framework como PyTorch o TensorFlow.



## Pasos Obligatorios


1. Carga y Preparación de Datos:

- Cargar el archivo gastro_reseñas_etiquetadas.csv (usando pandas).

- Preparar los datos para el modelo. Dado que es un problema multietiqueta, debes consolidar las columnas de etiquetas en un único formato que el modelo pueda entender (p.ej., un array de 8 dimensiones por cada reseña, como [1, 0, 0, 1, 0, 0, 0, 0]).

- Dividir los datos en conjuntos de entrenamiento, validación y prueba.

In [8]:
# Cargar el archivo gastro_reseñas_etiquetadas.csv
import pandas as pd
df = pd.read_csv('Data/gastro_reseñas_etiquetadas.csv')

# Mostrar las primeras filas del DataFrame
print(df.head())

  id_reseña                                       texto_reseña  \
0  gr_00001  El local estaba sucio y descuidado. En resumen...   
1  gr_00002  La comida estaba deliciosa. El local tiene una...   
2  gr_00003  Fue carísimo, un auténtico robo. El pollo esta...   
3  gr_00004  Era nuestra primera vez en este sitio.. Fuimos...   
4  gr_00005  Tengo sentimientos encontrados sobre este luga...   

   etiqueta_comida_pos  etiqueta_servicio_pos  etiqueta_ambiente_pos  \
0                    0                      0                      0   
1                    1                      1                      1   
2                    0                      0                      0   
3                    0                      0                      0   
4                    0                      0                      1   

   etiqueta_precio_pos  etiqueta_comida_neg  etiqueta_servicio_neg  \
0                    0                    0                      0   
1                    0        

In [9]:
# Preparar los datos: unificar columnas de etiqueta en un array de 8 dimensiones
import numpy as np
etiquetas = df[['etiqueta_comida_pos', 'etiqueta_servicio_pos', 'etiqueta_ambiente_pos', 'etiqueta_precio_pos', 'etiqueta_comida_neg', 'etiqueta_servicio_neg', 'etiqueta_ambiente_neg', 'etiqueta_precio_neg']].values
df['etiquetas_array'] = [list(map(int, etiqueta)) for etiqueta in etiquetas]
# Mostrar las primeras filas del DataFrame con la nueva columna
print(df[['id_reseña', 'etiquetas_array']].head())

  id_reseña           etiquetas_array
0  gr_00001  [0, 0, 0, 0, 0, 0, 1, 0]
1  gr_00002  [1, 1, 1, 0, 0, 0, 0, 0]
2  gr_00003  [0, 0, 0, 0, 1, 1, 0, 1]
3  gr_00004  [0, 0, 0, 0, 0, 0, 0, 0]
4  gr_00005  [0, 0, 1, 0, 0, 0, 0, 0]


In [10]:
# Dividir los datos en conjunto de entrenamiento y prueba
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print(f'Tamaño del conjunto de entrenamiento: {len(train_df)}')
print(f'Tamaño del conjunto de prueba: {len(test_df)}')

Tamaño del conjunto de entrenamiento: 8000
Tamaño del conjunto de prueba: 2000


2. Tokenización:

- Elegir e instanciar un tokenizador pre-entrenado (ej. DistilBertTokenizerFast o cualquier otro que hayais utilizado en clase).

- Tokenizar los texto_reseña para que el modelo Transformer pueda procesarlos (generando input_ids y attention_mask).

In [11]:
# Verificar nombres de columnas
print(df.columns.tolist())

['id_reseña', 'texto_reseña', 'etiqueta_comida_pos', 'etiqueta_servicio_pos', 'etiqueta_ambiente_pos', 'etiqueta_precio_pos', 'etiqueta_comida_neg', 'etiqueta_servicio_neg', 'etiqueta_ambiente_neg', 'etiqueta_precio_neg', 'etiquetas_array']


In [12]:
# Elegir e instanciar un tokenizador pre-entrenado (DistilBertTokenizerFast)
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenizar las reseñas generando input_ids y attention_masks
train_encodings = tokenizer(train_df['texto_reseña'].tolist(), padding='max_length', truncation=True, max_length=128)
test_encodings = tokenizer(test_df['texto_reseña'].tolist(), padding='max_length', truncation=True, max_length=128)

# Mostrar un ejemplo de tokenización
print(train_encodings['input_ids'][0])
print(train_encodings['attention_mask'][0])

[101, 8840, 28667, 8462, 8943, 4360, 1010, 2566, 2080, 9530, 24501, 2121, 12044, 1012, 27490, 2891, 14163, 2100, 4180, 2891, 9530, 2474, 16091, 12380, 1012, 3449, 2334, 5495, 2638, 14477, 25545, 21736, 3653, 9793, 3736, 1012, 3449, 2033, 5558, 2099, 18858, 10861, 2002, 4013, 9024, 2080, 4372, 2172, 2080, 5495, 8737, 2080, 1012, 16839, 8823, 16089, 26534, 14163, 2100, 5915, 2080, 1012, 11865, 2063, 14477, 2310, 27266, 2050, 29542, 20782, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

3. Configuración del Modelo:

- Cargar un modelo Transformer pre-entrenado (ej. DistilBertForSequenceClassification o cualquier otro que hayais utilizado en clase) desde la biblioteca transformers.

- Importante: Al cargar el modelo, deberás configurarlo específicamente para un problema de clasificación multietiqueta (multi-label), asegurándote de que la capa de salida tenga 8 neuronas (una por cada etiqueta posible) y utilice una función de activación y pérdida adecuadas (p.ej., Sigmoide en la salida y BCEWithLogitsLoss como función de pérdida).

In [13]:
# Cargar modelo de Transormer pre-entrenado: DistilBertForSequenceClassification
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=8, problem_type="multi_label_classification")

# Comprobar la arquitectura del modelo
print(model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


4. Entrenamiento (Fine-Tuning):

- Configurar los TrainingArguments (o un bucle de entrenamiento manual).

- Implementar el "fine-tuning" del modelo sobre el conjunto de datos de entrenamiento, monitorizando la pérdida en el conjunto de validación.

In [15]:
# Entrenamiento (Fine-Tuning)

from datasets import Dataset
from transformers import Trainer, TrainingArguments
import torch

# Preparar las etiquetas como arrays numpy para asegurar el formato correcto
train_labels = [[float(x) for x in label] for label in train_df['etiquetas_array'].tolist()]
test_labels = [[float(x) for x in label] for label in test_df['etiquetas_array'].tolist()]

# Convertir encodings y etiquetas a Dataset de Hugging Face
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

test_dataset = Dataset.from_dict({
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels
})

# Configurar el formato del dataset
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Configurar los TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',             # directorio para guardar los resultados
    num_train_epochs=1,                 # número de épocas (subir si no mejora y bajar si hay overfitting)
    per_device_train_batch_size=64,     # tamaño del batch para entrenamiento
    per_device_eval_batch_size=64,      # tamaño del batch para evaluación
    learning_rate=3e-5,                 # tasa de aprendizaje (más bajo si no converge y más alto si entrena lento)
    warmup_steps=50,                   # pasos de calentamiento
    weight_decay=0.01,                  # decaimiento del peso (aumentar si hay overfitting y disminuir si no hay overfitting)
    logging_dir='./logs',               # directorio para los logs
    logging_steps=20,                  # pasos entre logs
    eval_strategy="epoch",              # estrategia de evaluación
    save_strategy="epoch",              # estrategia de guardado
    load_best_model_at_end=True,        # cargar el mejor modelo al final
)

# Crear el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Entrenar el modelo
trainer.train()

# Guardar el modelo
model.save_pretrained('./modelo_gastro_final')
tokenizer.save_pretrained('./modelo_gastro_final')

Epoch,Training Loss,Validation Loss
1,0.265400,0.250254


('./modelo_gastro_final\\tokenizer_config.json',
 './modelo_gastro_final\\special_tokens_map.json',
 './modelo_gastro_final\\vocab.txt',
 './modelo_gastro_final\\added_tokens.json',
 './modelo_gastro_final\\tokenizer.json')

5. Evaluación:

- Una vez entrenado el modelo, usar el conjunto de prueba (datos que el modelo nunca ha visto) para evaluar su rendimiento.

- Calcular métricas apropiadas para clasificación multietiqueta, como el F1-Score (micro y macro), Precisión (micro) y Recall (micro).

Nota: No se puede usar 'accuracy' simple, ya que una predicción puede ser parcialmente correcta (acertar 3 de 8 etiquetas).

In [16]:
# Evaluar el modelo
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
import torch

predictions = trainer.predict(test_dataset)
logits = predictions.predictions

predicted_labels = (torch.sigmoid(torch.tensor(logits)) >= 0.5).int().numpy() # Umbral de 0.5 para clasificación multilabel

# Calcular métricas
f1 = f1_score(test_labels, predicted_labels, average='macro')
precision = precision_score(test_labels, predicted_labels, average='macro')
recall = recall_score(test_labels, predicted_labels, average='macro')

print(f"F1 Score: {f1}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(classification_report(test_labels, predicted_labels))

# Calcular micro F1, precision y recall
f1_micro = f1_score(test_labels, predicted_labels, average='micro')
precision_micro = precision_score(test_labels, predicted_labels, average='micro')
recall_micro = recall_score(test_labels, predicted_labels, average='micro')

print(f"Micro F1 Score: {f1_micro}")
print(f"Micro Precision: {precision_micro}")
print(f"Micro Recall: {recall_micro}")


c:\Users\ST09\Documents\GitHub\STEMIA\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


F1 Score: 0.8123396212209676
Precision: 0.8512518968371363
Recall: 0.7819742706792416
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       595
           1       0.95      0.97      0.96       623
           2       0.97      0.99      0.98       627
           3       0.99      0.99      0.99       637
           4       0.66      0.52      0.58       405
           5       0.67      0.55      0.61       381
           6       0.74      0.57      0.64       415
           7       0.83      0.66      0.74       432

   micro avg       0.89      0.83      0.86      4115
   macro avg       0.85      0.78      0.81      4115
weighted avg       0.88      0.83      0.85      4115
 samples avg       0.76      0.75      0.74      4115

Micro F1 Score: 0.8573232323232324
Micro Precision: 0.8922470433639947
Micro Recall: 0.8250303766707169


c:\Users\ST09\Documents\GitHub\STEMIA\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ST09\Documents\GitHub\STEMIA\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\ST09\Documents\GitHub\STEMIA\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitali

6. Función de Inferencia:

- Crear una función final llamada clasificar_reseña(texto) que reciba un string con una nueva reseña y devuelva una lista con los nombres de las etiquetas predichas (ej. ['Comida_Positivo', 'Servicio_Negativo']).

In [17]:
# Función de inferencia
def inferir_reseña(texto_reseña):
    # Cargar el modelo y el tokenizador guardados
    from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
    import torch

    modelo_cargado = DistilBertForSequenceClassification.from_pretrained('./modelo_gastro_final')
    tokenizador_cargado = DistilBertTokenizerFast.from_pretrained('./modelo_gastro_final')

    # Tokenizar la reseña
    encoding = tokenizador_cargado(texto_reseña, return_tensors='pt', padding='max_length', truncation=True, max_length=128)

    # Realizar la predicción
    with torch.no_grad():
        outputs = modelo_cargado(**encoding)
        logits = outputs.logits
        probabilidades = torch.sigmoid(logits).squeeze().numpy()

    # Aplicar umbral para obtener etiquetas binarias
    etiquetas_predichas = (probabilidades >= 0.5).astype(int)

    return etiquetas_predichas, probabilidades

# Ejemplo de uso de la función de inferencia
resania_ejemplo = "El ambiente del restaurante era acogedor, pero el servicio fue lento y la comida no estaba a la altura de mis expectativas."
etiquetas_predichas, probabilidades = inferir_reseña(resania_ejemplo)
print(f"Etiquetas predichas: {etiquetas_predichas}")
print(f"Probabilidades: {probabilidades}")

Etiquetas predichas: [0 0 0 0 0 0 0 0]
Probabilidades: [0.12766056 0.12475957 0.49528953 0.15224287 0.10678678 0.1413088
 0.13246131 0.11023088]
